# TODO

## Installing Libraries

In [2]:
%pip install numpy pandas scipy matplotlib

## Data download and inspection

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [12]:
!wget https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-Re.csv
!wget https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-data.csv
!wget https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-space.csv

--2023-10-12 16:56:01--  https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-Re.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1200009 (1.1M) [text/plain]
Saving to: ‘velprof-Re.csv’

velprof-Re.csv      100%[===================>]   1.14M  --.-KB/s    in 0.06s   

2023-10-12 16:56:01 (18.0 MB/s) - ‘velprof-Re.csv’ saved [1200009/1200009]

--2023-10-12 16:56:01--  https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting

Read Reynolds (the lables for our model)

In [15]:
pd.read_csv("velprof-Re.csv")

,# Re (-)
0,323590.756000
1,194866.844000
2,104938.956000
3,227933.606000
4,817618.639000
...,...
79995,647240.481000
79996,951.764062
79997,739653.106000
79998,676596.194000


Read the data file (the features for our model)

In [18]:
pd.read_csv("velprof-data.csv")["# mu (Pas)"]

0.000630     996.232835
0.001484    1117.370480
0.000372    1039.442220
0.000189    1045.145970
0.002309    1086.237000
               ...     
0.000527    1070.092390
0.000941    1041.655360
0.001010    1006.568710
0.000809    1001.212820
0.000663     926.350850
Name: # mu (Pas), Length: 80000, dtype: float64